# **Hand Gesture Recognition**
### **Anurag Bombarde**
### **Puja Kumari**


### **Project has  3D Conv models and Conv2D + RNN (GRU) models**

The objective of this projects is to build a hand gesture recognition model that can be hosted on a camera installed in a smart TV that can understand 5 gestures. Namely, leftwards hand movement to go to previous channel, rightward hand movement to go to next channel, upward hand movement to increase the volume, downward hand movement to decrease the volume and a palm gesture to pause playing the video.



In [1]:
#Checking the GPU Information
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Oct 20 09:02:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
pip install scipy==1.2.1

     |████████████████████████████████| 24.8 MB 1.4 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [3]:
import numpy as np
import os
# from scipy.misc.pilutil import imread
from scipy.misc import imread, imresize
import datetime


Setting the random seed so that the results don't vary drastically.

In [4]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf

tf.compat.v1.set_random_seed(30)

Unzipped data into the GDrive and reading from it by mounting it


In [5]:
# open('sample_data/README.md').readlines()
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
ls "/content/drive/MyDrive/Project_data/Project_data/"

train/  train.csv  val/  val.csv


Reading the folder names for training and validation. Also setting the `batch_size` here. 

In [7]:


train_doc = np.random.permutation(open('/content/drive/MyDrive/Project_data/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/drive/MyDrive/Project_data/Project_data/val.csv').readlines())
batch_size = 64 #Tried with batch size of 10 and 50 , it takes too long to execute 10 batchsize

## Generator
In the generator, going to preprocess the images as images are of 2 different dimensions as well as creating a batch of video frames. 

In [8]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    # print('folder_list=',folder_list)
    #List of images using image indexes as below to use for the sequence 
    img_idx = [0,1,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size)
        #Iterating over number of batches
        for batch in range(num_batches):
            # 18 as per img_idx is the number of images used for each video, (x,y) = (84,84) is the final size of the input images and 3 is the number of channels RGB
            batch_data = np.zeros((batch_size,18,84,84,3))
            # one hot representation of the output
            batch_labels = np.zeros((batch_size,5))
            # iterating over the batch_size
            for folder in range(batch_size):
                # read all the images in the folder
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])
                #  Iterating over the frames/images of a folder to read them in
                for idx,item in enumerate(img_idx):

                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    #cropping the images and resizing them. Note that the images are of 2 different shape 
                    image = imresize(image,(84,84)).astype(np.float32)
                    #normalising using min/max normalization and feeding in the image
                    batch_data[folder,idx,:,:,0] = (image[:,:,0] - np.min(image[:,:,0]))/(np.max(image[:,:,0])- np.min(image[:,:,0])) 
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.min(image[:,:,1]))/(np.max(image[:,:,1])- np.min(image[:,:,1])) 
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.min(image[:,:,2]))/(np.max(image[:,:,2])- np.min(image[:,:,2])) 
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels
        # Below code is for remaining data points which are left after full batches
        if (len(t)%batch_size) != 0:
            batch_data = np.zeros((len(t)%batch_size,18,84,84,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    #cropping the images and resizing them. Note that the images are of 2 different shape 
                    image = imresize(image,(84,84)).astype(np.float32)
                    #normalising using min/max normalization and feeding in the image
                    batch_data[folder,idx,:,:,0] = (image[:,:,0] - np.min(image[:,:,0]))/(np.max(image[:,:,0])- np.min(image[:,:,0]))
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.min(image[:,:,1]))/(np.max(image[:,:,1])- np.min(image[:,:,1]))
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.min(image[:,:,2]))/(np.max(image[:,:,2])- np.min(image[:,:,2]))

                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels

In [9]:
#Date time to save models with timestamp as experimenting with multiple models
curr_dt_time = datetime.datetime.now()
train_path = '/content/drive/MyDrive/Project_data/Project_data/train'
val_path = '/content/drive/MyDrive/Project_data/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
#Choosing epochs as 30 
num_epochs = 30
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


## Model
Creating 2 models using different functionalities that Keras provides those are Conv3D and Conv2D+RNN(GRU). Using `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. Using `TimeDistributed` while building a Conv2D + RNN model. Last layer is the softmax. 


*** Architecture 1 ***. **Convolution 3D Model**


In [10]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Dropout, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

###**** Model 1***##
# model = Sequential()
# model.add(Conv3D(32, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

# model.add(Conv3D(32, (3,3,3), strides=(1,1,1), padding='same'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
# model.add(Dropout(0.25))

# model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
# model.add(Dropout(0.25))

# model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

# model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.25))
# model.add(Dense(5, activation='softmax'))

##******************** With above model and adam optimizer below is the result
### ******.  Total params: 1,838,565
### *******. Trainable params: 1,838,181
#### ******.  Non-trainable params: 38 ********###
#### ***** Not performing good as below result is coming as best possible model with categorical/training accuracy as 64% and validation accuracy as 18%
# Epoch 28/30
# 11/11 [==============================] - 70s 7s/step - loss: 0.9562 - categorical_accuracy: 0.6425 - val_loss: 2.6069 - val_categorical_accuracy: 0.1800

# Epoch 00028: saving model to model_init_2021-10-1910_58_01.860195/model-00028-0.95618-0.64253-2.60688-0.18000.h5

### Hence experimenting with another model as below ##


###**** Model 2***##
nb_featuremap = [8,16,32,64]
nb_dense = [128,64,5]
nb_classes = 5
Input_shape = (18,84,84,3)
model = Sequential()
model.add(Conv3D(nb_featuremap[0], 
                 kernel_size=(5,5,5),
                 input_shape=Input_shape,
                 padding='same', name="conv1"))
model.add(Activation('relu'))
model.add(Conv3D(nb_featuremap[1], 
                 kernel_size=(3,3,3),
                 padding='same',name="conv2"))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Conv3D(nb_featuremap[2], 
                 kernel_size=(1,3,3), 
                 padding='same',name="conv3"))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Flatten())
model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(nb_dense[1], activation='relu'))
#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))


Compiling and printing model summary for Conv3D model





In [11]:

#Using adam optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 18, 84, 84, 8)     3008      
_________________________________________________________________
activation (Activation)      (None, 18, 84, 84, 8)     0         
_________________________________________________________________
conv2 (Conv3D)               (None, 18, 84, 84, 16)    3472      
_________________________________________________________________
activation_1 (Activation)    (None, 18, 84, 84, 16)    0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 9, 42, 42, 16)     0         
_________________________________________________________________
conv3 (Conv3D)               (None, 9, 42, 42, 32)     4640      
_________________________________________________________________
activation_2 (Activation)    (None, 9, 42, 42, 32)     0

Creating the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [12]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [13]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [14]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Fitting the model. This will start training the model and with the help of the checkpoints, it will save the model at the end of each epoch.

In [15]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/drive/MyDrive/Project_data/Project_data/train ; batch size = 64


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


Epoch 1/30
10/11 [==========================>...] - ETA: 5:33 - loss: 2.1256 - categorical_accuracy: 0.2172 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


11/11 [==============================] - 3742s 372s/step - loss: 2.1070 - categorical_accuracy: 0.2187 - val_loss: 1.6159 - val_categorical_accuracy: 0.2200

Epoch 00001: saving model to model_init_2021-10-2009_02_55.691325/model-00001-2.10697-0.21870-1.61595-0.22000.h5
Epoch 2/30
11/11 [==============================] - 59s 6s/step - loss: 1.5723 - categorical_accuracy: 0.2489 - val_loss: 1.6488 - val_categorical_accuracy: 0.1700

Epoch 00002: saving model to model_init_2021-10-2009_02_55.691325/model-00002-1.57226-0.24887-1.64878-0.17000.h5
Epoch 3/30
11/11 [==============================] - 61s 6s/step - loss: 1.4880 - categorical_accuracy: 0.3379 - val_loss: 1.6013 - val_categorical_accuracy: 0.2000

Epoch 00003: saving model to model_init_2021-10-2009_02_55.691325/model-00003-1.48804-0.33786-1.60126-0.20000.h5
Epoch 4/30
11/11 [==============================] - 59s 6s/step - loss: 1.3651 - categorical_accuracy: 0.4374 - val_loss: 1.6009 - val_categorical_accuracy: 0.2200

Epoch 00

Looks like model 2 of Conv3D archtecture above is performing good with 
11/11 [==============================] - 64s 6s/step - loss: 0.0724 - categorical_accuracy: 0.9834 - val_loss: 0.9730 - val_categorical_accuracy: 0.6800

i.e training accuracy of 97% and validation accuracy of 68%

*** Architecture 2***. **Convolution 2D+GRU Model**


In [16]:



from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, LSTM, Bidirectional


modelGRU = Sequential()
modelGRU.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2), activation='relu', padding='same'), input_shape=Input_shape))
modelGRU.add(TimeDistributed(Conv2D(32, (3,3), kernel_initializer="he_normal", activation='relu')))
modelGRU.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
 
modelGRU.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu')))
modelGRU.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu')))
modelGRU.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
 
modelGRU.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
modelGRU.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
modelGRU.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
 
modelGRU.add(TimeDistributed(Conv2D(256, (3,3), padding='same', activation='relu')))
modelGRU.add(TimeDistributed(Conv2D(256, (3,3), padding='same', activation='relu')))
modelGRU.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
 
modelGRU.add(TimeDistributed(Conv2D(512, (3,3), padding='same', activation='relu')))
modelGRU.add(TimeDistributed(Conv2D(512, (3,3), padding='same', activation='relu')))
modelGRU.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
 
modelGRU.add(TimeDistributed(Flatten()))
 
modelGRU.add(Dropout(0.5))
modelGRU.add(GRU(512, return_sequences=False, dropout=0.5))
modelGRU.add(Dense(5, activation='softmax'))




Compiling and printing model summary for Conv2D+GRU model 1


In [17]:
modelGRU.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (modelGRU.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 18, 42, 42, 32)    4736      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 18, 40, 40, 32)    9248      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 18, 20, 20, 32)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 18, 20, 20, 64)    18496     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 18, 20, 20, 64)    36928     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 18, 10, 10, 64)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 18, 10, 10, 128)  

Fitting GRU Model 1

In [18]:
modelGRU.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


Epoch 1/30
 9/11 [=======================>......] - ETA: 8s - loss: 1.6163 - categorical_accuracy: 0.1667 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


11/11 [==============================] - 64s 6s/step - loss: 1.6146 - categorical_accuracy: 0.1735 - val_loss: 1.6067 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2021-10-2009_02_55.691325/model-00001-1.61462-0.17345-1.60672-0.23000.h5
Epoch 2/30
11/11 [==============================] - 59s 6s/step - loss: 1.6091 - categorical_accuracy: 0.1916 - val_loss: 1.6084 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to model_init_2021-10-2009_02_55.691325/model-00002-1.60910-0.19155-1.60844-0.21000.h5
Epoch 3/30
11/11 [==============================] - 65s 7s/step - loss: 1.6111 - categorical_accuracy: 0.1946 - val_loss: 1.6067 - val_categorical_accuracy: 0.2100

Epoch 00003: saving model to model_init_2021-10-2009_02_55.691325/model-00003-1.61110-0.19457-1.60670-0.21000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/30
11/11 [==============================] - 59s 6s/step - loss: 1.6101 - categorical_acc

Looks like above model is not perforning well as validation and training data accuracy is not improving . Hence using another model as below

In [19]:
modelGRU2 = Sequential()

modelGRU2.add(TimeDistributed(Conv2D(nb_featuremap[0], (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=Input_shape))


modelGRU2.add(TimeDistributed(Conv2D(nb_featuremap[1], (3,3),padding='same', activation='relu')))
modelGRU2.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

modelGRU2.add(TimeDistributed(Conv2D(nb_featuremap[2], (3,3),padding='same', activation='relu')))
modelGRU2.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

modelGRU2.add(TimeDistributed(Conv2D(nb_featuremap[3], (2,2),padding='same', activation='relu')))
modelGRU2.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

modelGRU2.add(TimeDistributed(BatchNormalization()))
modelGRU2.add(Dropout(0.25))

modelGRU2.add(TimeDistributed(Flatten()))

modelGRU2.add(Dense(nb_dense[0], activation='relu'))
modelGRU2.add(Dropout(0.25))
modelGRU2.add(Dense(nb_dense[1], activation='relu'))
modelGRU2.add(Dropout(0.25))

## using GRU as the RNN model along with softmax as our last layer.
modelGRU2.add(GRU(128, return_sequences=False))
modelGRU2.add(Dense(nb_classes, activation='softmax')) # using Softmax as last layer
modelGRU2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (modelGRU2.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_16 (TimeDis (None, 18, 42, 42, 8)     224       
_________________________________________________________________
time_distributed_17 (TimeDis (None, 18, 42, 42, 16)    1168      
_________________________________________________________________
time_distributed_18 (TimeDis (None, 18, 21, 21, 16)    0         
_________________________________________________________________
time_distributed_19 (TimeDis (None, 18, 21, 21, 32)    4640      
_________________________________________________________________
time_distributed_20 (TimeDis (None, 18, 10, 10, 32)    0         
_________________________________________________________________
time_distributed_21 (TimeDis (None, 18, 10, 10, 64)    8256      
_________________________________________________________________
time_distributed_22 (TimeDis (None, 18, 5, 5, 64)     

Fitting GRU model 2

In [20]:
modelGRU2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


Epoch 1/30
 8/11 [====================>.........] - ETA: 13s - loss: 1.5213 - categorical_accuracy: 0.3281

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


11/11 [==============================] - 63s 6s/step - loss: 1.4610 - categorical_accuracy: 0.3635 - val_loss: 1.5862 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2021-10-2009_02_55.691325/model-00001-1.46099-0.36350-1.58623-0.23000.h5
Epoch 2/30
11/11 [==============================] - 61s 6s/step - loss: 1.0513 - categorical_accuracy: 0.5732 - val_loss: 1.5781 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to model_init_2021-10-2009_02_55.691325/model-00002-1.05132-0.57315-1.57811-0.21000.h5
Epoch 3/30
11/11 [==============================] - 62s 6s/step - loss: 0.8353 - categorical_accuracy: 0.6712 - val_loss: 1.5616 - val_categorical_accuracy: 0.2300

Epoch 00003: saving model to model_init_2021-10-2009_02_55.691325/model-00003-0.83528-0.67119-1.56163-0.23000.h5
Epoch 4/30
11/11 [==============================] - 59s 6s/step - loss: 0.5933 - categorical_accuracy: 0.7647 - val_loss: 1.5523 - val_categorical_accuracy: 0.3200

Epoch 00004:

**Conclusion** Comparing both the models Conv3D and GRU+Conv2D model . 2nd model of GRU+Conv2D is doing well with from model  "model-00024-0.00130-1.00000-0.85996-0.73000.h5" with 100% training accuracy and 73% validation accuracy
